In [1]:
import music21
import glob
import numpy as np
import os
from music21 import converter, instrument, note, chord, stream, tempo, duration
import matplotlib.pyplot as plt
import pickle
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
# from music_generator.midi import parseNote, get_notes, render_midi
from tensorflow.keras.optimizers import RMSprop

In [2]:
!pip install git+https://github.com/UlyssePiediscalzi/music-generator.git

  Cloning https://github.com/UlyssePiediscalzi/music-generator.git to /private/var/folders/3k/_7hb2yqx1f78znggrmfpm3100000gn/T/pip-req-build-ianoo4qs
  Running command git clone --filter=blob:none -q https://github.com/UlyssePiediscalzi/music-generator.git /private/var/folders/3k/_7hb2yqx1f78znggrmfpm3100000gn/T/pip-req-build-ianoo4qs
  fatal: unable to access 'https://github.com/UlyssePiediscalzi/music-generator.git/': Couldn't connect to server
ERROR: Command errored out with exit status 128: git clone --filter=blob:none -q https://github.com/UlyssePiediscalzi/music-generator.git /private/var/folders/3k/_7hb2yqx1f78znggrmfpm3100000gn/T/pip-req-build-ianoo4qs Check the logs for full command output.


# Importing

In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
files = glob.glob("../raw_data/classic_piano/chopin/*.mid") # get the filepaths
files

['../raw_data/classic_piano/chopin/chpn_op23.mid',
 '../raw_data/classic_piano/chopin/chpn-p19.mid',
 '../raw_data/classic_piano/chopin/chpn_op7_2.mid',
 '../raw_data/classic_piano/chopin/chpn-p18.mid',
 '../raw_data/classic_piano/chopin/chpn-p24.mid',
 '../raw_data/classic_piano/chopin/chpn_op7_1.mid',
 '../raw_data/classic_piano/chopin/chpn-p23.mid',
 '../raw_data/classic_piano/chopin/chpn-p9.mid',
 '../raw_data/classic_piano/chopin/chpn-p8.mid',
 '../raw_data/classic_piano/chopin/chpn-p22.mid',
 '../raw_data/classic_piano/chopin/chpn-p20.mid',
 '../raw_data/classic_piano/chopin/chpn-p21.mid',
 '../raw_data/classic_piano/chopin/chp_op18.mid',
 '../raw_data/classic_piano/chopin/chpn_op35_4.mid',
 '../raw_data/classic_piano/chopin/chpn_op33_2.mid',
 '../raw_data/classic_piano/chopin/chp_op31.mid',
 '../raw_data/classic_piano/chopin/chpn_op25_e4.mid',
 '../raw_data/classic_piano/chopin/chpn_op35_2.mid',
 '../raw_data/classic_piano/chopin/chpn_op25_e1.mid',
 '../raw_data/classic_piano/ch

In [3]:
def get_notes(artist, n_files):
  """ Get all the notes and chords from the midi files in the ./midi_songs directory """
  notes = []
  quarter_length= []
  files = glob.glob(f"../raw_data/classic_piano/{artist}/*.mid") # get the filepaths

  for file in files[:n_files]: 
    midi = converter.parse(file)

    print("Parsing %s" % file)

    notes_to_parse = None

    try: # file has instrument parts
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts.recurse() 
    except: # file has notes in a flat structure
        notes_to_parse = midi.flat.notes

    for element in notes_to_parse:
      if isinstance(element, note.Note):
        notes.append(str(element.pitch))
        # notes.append(str(element.pitch)} for {element.duration.quarterLength}')  #appending notes + duration to single string

        # quarter_length.append(float(element.duration.quarterLength)) #appending all notes durations to quarter_length
      elif isinstance(element, chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))

        # c = '.'.join(f"{n}" for n in element.normalOrder)       #appending chord + duration to single string
        # notes.append(f"{c} for {element.duration.quarterLength}")

        # quarter_length.append(float(element.duration.quarterLength))  #appending all chords durations to quarter_length
  return notes

In [4]:
notes = get_notes('chopin', 15)
notes

/Users/will/.pyenv/versions/3.8.12/envs/music-generator/lib/python3.8/site-packages/music21/midi/translate.py:785: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel=None, data=b'Copyright \xa9 2003 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(


Parsing ../raw_data/classic_piano/chopin/chpn_op23.mid


/Users/will/.pyenv/versions/3.8.12/envs/music-generator/lib/python3.8/site-packages/music21/midi/translate.py:785: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel=None, data=b'Copyright \xa9 2002 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(


Parsing ../raw_data/classic_piano/chopin/chpn-p19.mid
Parsing ../raw_data/classic_piano/chopin/chpn_op7_2.mid
Parsing ../raw_data/classic_piano/chopin/chpn-p18.mid
Parsing ../raw_data/classic_piano/chopin/chpn-p24.mid
Parsing ../raw_data/classic_piano/chopin/chpn_op7_1.mid
Parsing ../raw_data/classic_piano/chopin/chpn-p23.mid


/Users/will/.pyenv/versions/3.8.12/envs/music-generator/lib/python3.8/site-packages/music21/midi/translate.py:785: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 1997 by Bernd Krueger'>; getting generic Instrument
  warnings.warn(


...

/Users/will/.pyenv/versions/3.8.12/envs/music-generator/lib/python3.8/site-packages/music21/midi/translate.py:785: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel=None, data=b'Copyright \xa9 1997 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(


Parsing ../raw_data/classic_piano/chopin/chp_op18.mid


/Users/will/.pyenv/versions/3.8.12/envs/music-generator/lib/python3.8/site-packages/music21/midi/translate.py:785: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=4, channel=None, data=b'Copyright \xa9 2002 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(


Parsing ../raw_data/classic_piano/chopin/chpn_op35_4.mid
Parsing ../raw_data/classic_piano/chopin/chpn_op33_2.mid


['C3',
 'C2',
 'E-3',
 'E-2',
 'G#3',
 'G#2',
 'B-3',
 'B-2',
 'C4',
 'C3',
 'G#3',
 'G#2',
 'E-4',
 'E-3',
 'B-4',
 'B-3',
 'C5',
 'C4',
 'G#4',
 'G#3',
 'E-5',
 'E-4',
 'B-5',
 'B-4',
 'C6',
 'C5',
 'G5',
 'G4',
 'B-5',
 'B-4',
 'G#5',
 'G#4',
 'G5',
 'G4',
 'F#5',
 'F#4',
 'F#5',
 'F#4',
 'G5',
 'G4',
 'F#5',
 'F#4',
 'F5',
 'F4',
 'F#5',
 'F#4',
 'A5',
 'A4',
 'G5',
 'G4',
 'E-5',
 'E-4',
 'E-5',
 'E-4',
 'D5',
 'D4',
 'F5',
 'F4',
 'E-5',
 'E-4',
 'D5',
 'D4',
 'D5',
 'D4',
 'C5',
 '0.3.7',
 'G4',
 'D3',
 'G3',
 'B-4',
 'E-4',
 'B-4',
 'D2',
 'C4',
 'D4',
 'F#4',
 'B-4',
 'A4',
 'G4',
 '10.2',
 'G2',
 '10.2',
 'G2',
 'D5',
 '2.7',
 'B-3',
 '2.7',
 'B-3',
 'C5',
 '0.3.7',
 'A3',
 '0.3.7',
 'A3',
 'C4',
 'D4',
 'D2',
 'F#4',
 'B-4',
 'D2',
 'A4',
 'G4',
 '10.2',
 'G2',
 '10.2',
 'G2',
 'E4',
 '7.9.1',
 'A2',
 '7.9.1',
 'A2',
 'F#4',
 '2.6.9',
 'D2',
 '2.6.9',
 'D2',
 'C4',
 'D4',
 'D3',
 'F#4',
 'B-4',
 'D3',
 'A4',
 'G4',
 '10.2',
 'G2',
 '10.2',
 'G2',
 'G5',
 '7.10.1',
 'E4',
 '7

# Preproc 

In [5]:
sequence_length = 40

In [6]:
def preproc_input(notes, sequence_length):
  n_vocab = len(set(notes))
  pitchnames = sorted(set(item for item in notes))
  note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
  #get the sequences for input and output
  network_input = []
  network_output = []
  for i in range(0, len(notes) - sequence_length, 1): 
      sequence_in = notes[i:i + sequence_length] # create sequences of sequence_length notes/chords
      sequence_out = notes[i + sequence_length] # the output will be the next note in the sequence
      network_input.append([note_to_int[char] for char in sequence_in]) # append to the network_input the feature sequence of notes
      network_output.append(note_to_int[sequence_out]) # append to the network_output the output note
  n_patterns = len(network_input)
  network_input = np.reshape(network_input, (n_patterns, # 533 
                                              sequence_length, # 50
                                              1)) # 1 feature (the mapped note or chord)
  # normalize input
  network_input = network_input / float(n_vocab)  
  # categorize output
  network_output = to_categorical(network_output) # transforms the output note/chord into categorical features
  return n_vocab, pitchnames,  note_to_int, n_patterns, network_input, network_output

In [7]:
n_vocab, pitchnames,  note_to_int, n_patterns, network_input, network_output = preproc_input(notes, sequence_length)

In [14]:
n_vocab

286

# Model

In [8]:
model = models.Sequential()

model.add(layers.LSTM(512, return_sequences=True, input_shape=(60,1))) # network_input.shape = (533,50,1)
model.add(layers.LSTM(256))
model.add(layers.Dropout(rate=0.2))
model.add(layers.Dense(35, activation='relu'))
model.add(layers.Dense(286, activation='softmax')) # 71 is the n_vocab, the total number of different notes/chords
# the model was using n_vocab which was equal to 71, but out y only has 70 columns so I needed to adjust it

model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.001))

model.summary()

2021-11-29 16:52:06.018015: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 60, 512)           1052672   
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               787456    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 35)                8995      
_________________________________________________________________
dense_1 (Dense)              (None, 286)               10296     
Total params: 1,859,419
Trainable params: 1,859,419
Non-trainable params: 0
_________________________________________________________________


# Training the model

In [9]:
model.fit(network_input, network_output, epochs=2, batch_size=32)

Epoch 1/2


2021-11-29 16:52:06.639281: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)


517/517 [==============================] - 304s 584ms/step - loss: 4.8874
Epoch 2/2
517/517 [==============================] - 292s 564ms/step - loss: 4.7935


# Saving or Loading the Model

In [12]:
from tensorflow.keras.models import load_model

model.save('../raw_data/model_w.h5')

In [13]:
from tensorflow.keras.models import load_model
model = load_model('../raw_data/model_w.h5')


# Func to extract notes and chords 

In [32]:
import random
def chords_n_notes(Snippet):
    Melody = []
    offset = 0 #Incremental
    for i in Snippet:
        #If it is chord
        if ("." in i or i.isdigit()):
            chord_notes = i.split(".") #Seperating the notes in chord
            notes = [] 
            for j in chord_notes:
                inst_note=int(j)
                note_snip = note.Note(inst_note)            
                notes.append(note_snip)
                chord_snip = chord.Chord(notes)
                chord_snip.offset = offset
                Melody.append(chord_snip)
        # pattern is a note
        else: 
            note_snip = note.Note(i)
            note_snip.offset = offset
            Melody.append(note_snip)
        # increase offset each iteration so that notes do not stack
        offset += 0.5
    Melody_midi = stream.Stream(Melody)   
    return Melody_midi

# Melody_Snippet = chords_n_notes(notes[:10])

# show(Melody_Snippet)

# Func to predict output

In [33]:
def Malody_Generator(Note_Count):
    seed = network_input[np.random.randint(0,len(network_input)-1)]
    Music = ""
    Notes_Generated=[]
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    for i in range(Note_Count):
        seed = seed.reshape(1,40,1)
        prediction = model.predict(seed, verbose=0)[0]
        prediction = np.log(prediction) #diversity
        exp_preds = np.exp(prediction) / 1
        prediction = exp_preds / np.sum(exp_preds)
        index = np.argmax(prediction)
        index_N = index/ float(n_vocab)   
        Notes_Generated.append(index)
        Music = [int_to_note[char] for char in Notes_Generated]
        # print(seed)
        # print(len(seed))
        seed = np.insert(seed[0],len(seed[0]),index_N)
        seed = seed[1:]
    #Now, we have music in form or a list of chords and notes and we want to be a midi file.
    Melody = chords_n_notes(Music)
    Melody_midi = stream.Stream(Melody)   
    return Music,Melody_midi


#getting the Notes and Melody created by the model
Music_notes, Melody = Malody_Generator(200)
# show(Melody)

# Write a new midi

In [34]:
Melody.write('midi', fp='../raw_data/test_60_10t_testpb.mid')

'../raw_data/test_60_10t_testpb.mid'